# Hyperparameter Tuning using HyperDrive

*TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.*

In [1]:
import train_xgb # The module for loading external data and training an XGB model
import os
import pandas as pd
import numpy as np
import json
import ast
import pickle
import joblib

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Workspace, Dataset, Experiment, Model, Environment, ScriptRunConfig
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.widgets import RunDetails

from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, loguniform, choice

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

quick-starts-ws-154647
aml-quickstarts-154647
southcentralus
61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30


In [3]:
# Create compute cluster
# Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           min_nodes=1,
                                                           max_nodes=5)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset

*TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.*

The Ames Housing dataset is downloaded from Kaggle.com and registered to the workspace. For HyperDrive child runs is used CSV file in local folder to prevent "Too many requests" error.

In [4]:
# # Try to load the dataset from the workspace. Otherwise, load if from Kaggle
found = False
ds_key = 'Ames-housing-dataset'
ds_desc = 'Ames Housing training data.'

if ds_key in ws.datasets.keys():
    found = True
    dataset = ws.datasets[ds_key]
    print(f'Found registered {ds_key}, use it.')
    
if not found:
    train, test = train_xgb.load_data_clean(source='kaggle')
    print(f"train.shape = {train.shape}, test.shape = {test.shape}")
    # Register the train dataset
    blob = ws.get_default_datastore()
    dataset = TabularDatasetFactory.register_pandas_dataframe(train, blob, name=ds_key, description=ds_desc)

Found registered Ames-housing-dataset, use it.


In [ ]:
# Test the train script
# ! python train_xgb.py

## Hyperdrive Configuration

*TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.*

The HyperDrive run optimizes the XGBoostRegressor which has been the best performing model in the AutoML voting ensemble. The models using gradient tree boosting have usually better performance because they combine predictions of several base estimators.

Hyperparameters of XGBoostRegressor tuned are:
+ `learing_rate` alias `eta` is the step size shrinkage used in update to prevent overfiffing, its range is [0,1] and for search are used values in interval [0.01, 0.2] with log scale,
+ `gamma` specifies the minimum loss reduction required to make a further partition on a leaf node of the tree; the larger `gamma` is, the more conservative the algorithm will be; its range is [0,∞] and for search is used interval [0,9]
+ `max_depth` is maximum depth of a tree, increasing this value will make the model more complex and more likely to overfit; its range is [0,∞] and for search are used values 3, 5, and 7.

Other parameters are left with their default values except a learning parameter `objective` which takes value `reg:squarederror`.

For hyperparametr tuning itself, the pipeline uses the random hyperparameter sampler, as it supports discrete and continuous hyperparameters and also early termination of low-performance runs, which was enabled by the bandit policy. Benefit of both these design decisions is lower costs of the experiments.

In [5]:
# Choose a name for an experiment
experiment_name = 'Ames-housing-hdr'

experiment=Experiment(ws, experiment_name)

In [6]:
%%writefile conda_env.yml

dependencies:
- python=3.6.2
- pip:
  - inference-schema
  - azureml-defaults==1.32.0
- numpy>=1.16.0,<1.19.0
- pandas==0.25.1
- scikit-learn==0.22.1
- py-xgboost<=0.90
channels:
- anaconda
- conda-forge

Writing conda_env.yml


In [7]:
# Define an Azure ML environment
# Dependencies are the same as for AutoML experiment
env = Environment.from_conda_specification(name='env', file_path='conda_env.yml')

# Configure the training job
src = ScriptRunConfig(source_directory=".",
                     script='train_xgb.py',
                     #arguments=['--learning_rate', 0.01, '--gamma', 5, '--max_depth', 5], # Just for testing
                     compute_target=cpu_cluster,
                     environment=env)

In [ ]:
# Test the script run config
# run = experiment.submit(src)

In [8]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

#TODO: Create the different params that you will be using during training
# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--learning_rate': loguniform(-4.6, -1.6), # results in [0.01, 0.2]
        '--gamma': uniform(0, 9), 
        '--max_depth': choice(3, 5, 7)
    }
)

#TODO: Create your estimator and hyperdrive config
# src - see above

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                    hyperparameter_sampling=ps,
                                    policy=policy,
                                    primary_metric_name='r2_score',
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=20,
                                    max_concurrent_runs=4,
                                    max_duration_minutes=30)

In [9]:
#TODO: Submit your experiment
hdr = experiment.submit(config=hyperdrive_config)

## Run Details

*OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?*

Prediction of a sale price based on parameters of a house is an example of a **regression** which is a supervised machine learning task. [**XGBoostRegressor**](https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn) is an example of tree-based algorithms. It is an accurate and effective off-the-shelf procedure that supports a number of different loss functions and is developed with both deep consideration in terms of systems optimization and principles in machine learning. The models using gradient tree boosting have usually better performance because they combine predictions of several base estimators.

*TODO: In the cell below, use the `RunDetails` widget to show the different experiments.*

In [10]:
# Show run details with the widget.
RunDetails(hdr).show()
hdr.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_6c27daf6-f58e-4f0b-acda-6e1db235186b
Web View: https://ml.azure.com/runs/HD_6c27daf6-f58e-4f0b-acda-6e1db235186b?wsid=/subscriptions/61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30/resourcegroups/aml-quickstarts-154647/workspaces/quick-starts-ws-154647&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-08-16T09:47:51.765025][API][INFO]Experiment created<END>\n""<START>[2021-08-16T09:47:52.311828][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-08-16T09:47:52.497257][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_6c27daf6-f58e-4f0b-acda-6e1db235186b
Web View: https://ml.azure.com/runs/HD_6c27daf6-f58e-4f0b-acda-6e1db235186b?wsid=/subscriptions/61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30/resourcegroups/aml-quickstarts-154647/workspaces/quick-starts-ws-154647&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



{'runId': 'HD_6c27daf6-f58e-4f0b-acda-6e1db235186b',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-08-16T09:47:51.582075Z',
 'endTimeUtc': '2021-08-16T10:07:25.716391Z',
 'properties': {'primary_metric_config': '{"name": "r2_score", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '1d782f01-523b-4ec5-b7ba-e2557e18fc25',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1055-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.32.0',
  'score': '0.9130778329488691',
  'best_child_run_id': 'HD_6c27daf6-f58e-4f0b-acda-6e1db235186b_15',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg154647.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_6c27daf6-f58e-4f0b-acda-6e1db235186b/azureml-logs/hyperdrive.txt?sv=2019-07-07&sr=b&

## Best Model

*TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.*

In [11]:
# Get your best run and save the model from that run.
best_hdr_run = hdr.get_best_run_by_primary_metric()
print(best_hdr_run)

Run(Experiment: Ames-housing-hdr,
Id: HD_6c27daf6-f58e-4f0b-acda-6e1db235186b_15,
Type: azureml.scriptrun,
Status: Completed)


In [12]:
best_hdr_metrics = best_hdr_run.get_metrics()
best_hdr_metrics

{'Learning rate': 0.05776444084600065,
 'Gamma': 8.12061751815267,
 'Maximum depth': 7.0,
 'r2_score': 0.9130778329488691}

In [13]:
# Assumption: The AutoML run already done
with open('aml-outputs/best_aml_metrics.json', 'r') as file:
    best_aml_metrics = json.load(file)
print(f'AutoML\'s r2_score:\t{best_aml_metrics["r2_score"]}')
print(f'HDR-XGB\'s r2_score:\t{best_hdr_metrics["r2_score"]}')

AutoML's r2_score:	0.8942710428327086
HDR-XGB's r2_score:	0.9130778329488691


In [15]:
details = best_hdr_run.get_details()

# Save metrics and details for ex-post examination
os.makedirs('./hdr-outputs', exist_ok=True)
with open('hdr-outputs/best_hdr_metrics.json', 'w') as file:
    json.dump(best_hdr_metrics, file)
with open('hdr-outputs/best_hdr_details.txt', 'w') as file:
    file.write(str(details))

In [17]:
# Check the path to the model
for i,n in enumerate(best_hdr_run.get_file_names()):
    print(i,n)

0 azureml-logs/55_azureml-execution-tvmps_2c74c7815ff195280a4e09c6c749f7895640459993a696119bf6cc0d0013ea1b_d.txt
1 azureml-logs/65_job_prep-tvmps_2c74c7815ff195280a4e09c6c749f7895640459993a696119bf6cc0d0013ea1b_d.txt
2 azureml-logs/70_driver_log.txt
3 azureml-logs/75_job_post-tvmps_2c74c7815ff195280a4e09c6c749f7895640459993a696119bf6cc0d0013ea1b_d.txt
4 azureml-logs/process_info.json
5 azureml-logs/process_status.json
6 logs/azureml/97_azureml.log
7 logs/azureml/job_prep_azureml.log
8 logs/azureml/job_release_azureml.log
9 outputs/model.pkl


In [20]:
#TODO: Save the best model
best_hdr_run.download_file(best_hdr_run.get_file_names()[9], output_file_path='./hdr-outputs/')

## Model Deployment

Remember you have to deploy only one of the two models you trained. Perform the steps in the rest of this notebook only if you wish to deploy this model.

*TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.*

In [22]:
# Register the best model
model = Model.register(ws, model_path='hdr-outputs/model.pkl', model_name='Ames-Housing-XGB-Model', tags=best_hdr_metrics)
print(model.name, model.id, model.version, sep='\t')

Registering model Ames-Housing-XGB-Model
Ames-Housing-XGB-Model	Ames-Housing-XGB-Model:1	1


In [23]:
from azureml.core.webservice import AciWebservice
aciconfig = AciWebservice.deploy_configuration(cpu_cores=1,
                                              memory_gb=1,
                                              tags={"data" : "Kaggle", "method" : "XGB"},
                                              description="Predict Ames Housing Prices",
                                              auth_enabled=True,
                                              enable_app_insights=True)

In [24]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

env = Environment("project-env")
cd = CondaDependencies('conda_env.yml')
env.python.conda_dependencies = cd
# Register environment to re-use later
env.register(workspace=ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20210615.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "project-env",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-forge"
  

In [ ]:
%%time
import uuid
from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core import Workspace
from azureml.core.model import Model

ws = Workspace.from_config()
model = Model(ws, 'Ames-Housing-XGB-Model')

myenv = Environment.get(workspace=ws, name="project-env", version="1")

inference_config = InferenceConfig(entry_script="entry_script.py", environment=myenv)

service_name = 'ames-housing-xgb-' + str(uuid.uuid4())[:4]
service = Model.deploy(workspace=ws,
                      name=service_name,
                      models=[model],
                      inference_config=inference_config,
                      deployment_config=aciconfig)

service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-08-16 10:12:26+00:00 Creating Container Registry if not exists.
2021-08-16 10:12:26+00:00 Registering the environment.
2021-08-16 10:12:28+00:00 Use the existing image.
2021-08-16 10:12:28+00:00 Generating deployment configuration.
2021-08-16 10:12:30+00:00 Submitting deployment to compute.
2021-08-16 10:12:35+00:00 Checking the status of deployment ames-housing-xgb-badc.

In [ ]:
print(service.get_logs())

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()

In [ ]:
# Prepare data for request
_ , test = train_xgb.load_data_clean()
test = train_xgb.label_encode(test)
data = {'data': test.head().to_dict(orient='list')}

# Replace the next cell with the code from 'Consume' tab of the endpoint
# and delete 'data = {}' assignment as data is defined in this cell!  

In [ ]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

body = str.encode(json.dumps(data))

url = 'http://0680a4cc-0cb7-4e4e-a060-c506eaeb245b.southcentralus.azurecontainer.io/score'
api_key = 'SATMLdOCE7lqUBgY2afQSvxfHfl05ouL' # Replace this with the API key for the web service
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(json.loads(error.read().decode("utf8", 'ignore')))

In [ ]:
# Decode the result for subsequent processing
import ast
ast.literal_eval(json.loads(result))['result']

*TODO: In the cell below, print the logs of the web service and delete the service*

In [ ]:
print(service.get_logs())
# Clean up resources
service.delete()
cpu_cluster.delete()